# Guía Efecto Aleatorio

**Ejercicio 5**

$$y_{ijk} = \mu + paciente_i + corida_j +(paciente:corrida)_{ij} + \epsilon_{ijk}$$

Escribir cada efecto aleatorio por separado

```
m = lmer(Y~1 + (1|paciente) + (1|corrida) + (1|paciente:corrida))
```

En efectos aleatorios tenemos dos vias para analizar:

- máxima verosimilitud ... REML -> **lmer**: este recomienda el profesor
- momentos -> tabla ANOVA, si lo queremos ver por este lado aov( Error(efectos_aleatorio)), otro un poco mejor es usar un paquete EMSaov (ver en apunte de la UBA). 
    ```
    # solamente pone la interaccion, type le dice el tipo de efecto, R significa random, 
    # tener cuidado que los factores deben estar dentro del dataframe
    datos$fpaciente = as.factor(paciente)
    datos$fcorrida = as.factor(corrida)
    # otra forma si los cargas a mano es armarte el dataframe> datos = data.frame(Y:..., A:..., B:...)
    m = EMSanova(Y~fpaciente + fcorrida, type = c('R', 'R'), data = datos) 
    # esto te da los cuadrados medios esperados
    print(m) 
    ```


In [ ]:
# cargar los datos
datos = read.csv("")

# REML por defecto es true, cuando son efectos aleatorios da lo mismo que sea true o false, el tema es cuando hay efecto
# fijo, ahi si necesito ponerlo en FALSE
m = lmer(colesterol~1 + (1|fpaciente) + (1|fcorrida) + (1|fcorrida:fpaciente)) 
# nos lo reporta en el mismo orden que lo ingresamos

# al poner REML en false hace el test con chi2, pero aca no hace falta
confint(m)

# tenemos que hacerlo para saber 
# al estadistico hay que calcularlo a pata con efectos aleatorios

# .sigma nos da IC para el error
# .sig01 nos dice que no es significativa pq da 0 el limite inferior, por que esta en el borde y no en el intervalo? 
# pq la varianza no tiene valores negativos
# el orden que aparecen esos sigma01, 02,... tiene que ver con el orden que nos muestra el summary
summary(m)

m0 = lmer(colesterol~1+ (1|fpaciente) + (1|fcorrida))
# me da un pvalor grande entonces me puedo quedar con el modelo mas chico
# comparamos dos modelos anidados pero uno acotado
anova(m, m0)

# ese anova esta testeando 
# - H0: sigma2AB = 0
# - H1: sigma2AB > 0
# como nos da pvalor grande debemos aceptar H0, que es coherente con lo que vimos en los IC para sigmaAB

# cuando uno tiene un m1 y un m2 ajustado con lmer es pq tenes si o si efectos aleatorios, la forma de compararlo es
# anova(m1, m0) # como esta ajustado con lmer usa chi2, el problema es que es aproximado para n chico
# /podemos entonces hacer algo mejor que eso_
# para testear efectos aleatorios lo que podemos usar es una funcion que se llama 
# exactLRT(m1, m0) # no utiliza la aproximacion de chi2, el drama es que un monton de veces no funciona, y que funciona
# solamente para un efecto aleatorio
# lo que el profesor recomienda es:
# para efectos aleatorios, usar el test es usar el IC que nos devuelve confint !!!!
# para efectos fijos, el test que debemos hacer es con lmer pero usando REML = FALSE, luego usar igual anova(m1, m0)
# pero tenemos el mismo drama de chi2 pq puede tener poca potencia
# entonces se puede utilizar una library(pbkrtest)
# poniendo
# KRmodcomp(m1, m0)

# las conclusiones del ejercicio es que la corrida es significativa, lo que no es significativa es la interacción
# es despreciable la interaccion entre el paciente y la corrida pero NO la corrida sola

# Guía Efecto Mixto

- ``Practica_Mixtos.pdf``

**Ejercicio 3:** 

- Anidado: el operario esta anidado en la disposicion (d), 
- Cruzados: entre la programacion (p) y la disposición 
- Efectos fijos: disposicion y programación, son determinados, me intersa saber que combinación entre estas. Si fuese aleatorio esto sería un despelote la fábrica. 
- Aleatorio: operario (o)

## Modelo tentativo:

$$y_{ijk} = \mu + d_i + p_j + (dp)_{ij} + O_{k(i)} + \epsilon_{ijk}$$

## Análisis exploratorio

Gráfico de interacción entre los efectos fijos

```
interaction.plot(fdisposicion, fprogramacion, tiempo) # podria parecer que la interaccion es debil

# pero si graficamos el otro vemos que tenemos la misma tendencia, asi que nos esta diciendo por mas que se corte
# que no hay mucha interaccion
interaction.plot(fprogramacion, fdisposicion, tiempo) # podria parecer que la interaccion es debil

# osea, ambos sugieren que hay interacción debil

```

Ajustemos el modelo ahora

```
m1 = lmer(tiempo~fdisposicion*fprogramacion + (1|foperador%in%fdisposicion))
confint(m1)
```

Vemos que los IC contienen al cero entonces eso nos dice que no es importante

Ahora lo que queremos testear es efecto fijo, ahora usamos REML=FALSE

```
m1.ml =  lmer(tiempo~fdisposicion*fprogramacion + (1|operador%in%disposicion), REML=FALSE)
m0.ml =  lmer(tiempo~fdisposicion+fprogramacion + (1|operador%in%disposicion), REML=FALSE)

anova(m1.ml, m0.ml)
```

interaccion no parece significativa, podemos hacer comparaciones multiples por separado

Ahora podemos hacer
```
library(pbk...)
KRmodcomp(m1.ml, m0.ml)
m = lmer(tiempo~fdisposicion*fprogramacion + (1|operador%in%disposicion), REML=FALSE)

shapiro.test(residuals(m))

boxplot(tiempo~fdisposicion)
boxplot(tiempo~fprogramacion)

library(multcomp)
m.fita = glht(m , linfct=mcp(fprogramacion="Tukey"))
plot(m.fija)

m.fita = glht(m , linfct=mcp(fdisposicion="Tukey"))
plot(m.fija)

# aca lo hago sin considerar el termino aleatorio porque no es significativo
mm = aov(tiempo~fprogramacion+fdisposicion)
summary(mm)

# podria hacer comparaciones sin tener en cuenta el efecto aletatorio
# vamos a ver que no cambia mucho si hacemos las comparaciones con este modelo
TukeyHSD(mm)
```

Uno podria ver HSU para ver si la uno o la 3 son diferentes y si una conviene mas que la otra. En analisis uno lo puede hacer sobre uno de los factores porque el analisis no dio significativo.


In [ ]:
datos = read.csv()